In [ ]:
import sys
from typing import Callable, Self

sys.path.append('../utils')
from pyutils import *

In [ ]:
class ElfComputer:
    def __init__(self, r_a: int = 0, r_b: int = 0, r_c: int = 0):
        self.r_a = r_a
        self.r_b = r_b
        self.r_c = r_c
        self.inst_ptr: int = 0
        self.inst_set: list[Callable] = [
            self.adv, self.bxl, self.bst, self.jnz, self.bxc, self.out, self.bdv, self.cdv
        ]
        self.program: list[int] = []
        self.stdout: list[int] = []

    @classmethod
    def from_str(cls, initstr: str) -> Self:
        inst = cls()
        for line in initstr.split('\n'):
            if line.startswith('Register A:'):
                inst.r_a = int(line.split(':')[1].strip())
            elif line.startswith('Register B:'):
                inst.r_b = int(line.split(':')[1].strip())
            elif line.startswith('Register C:'):
                inst.r_c = int(line.split(':')[1].strip())
            elif line.startswith('Program:'):
                inst.program = list(map(int, line.split(':')[1].split(',')))
        return inst
    
    def get_combo_op(self, val: int) -> int:
        match val:
            case 0 | 1 | 2 | 3: return val
            case 4: return self.r_a
            case 5: return self.r_b
            case 6: return self.r_c
            case 7: raise ValueError('Attempt to use reserved combo operand 7')

    def run_program(self, must_copy: bool = False):
        while True:
            copy_success = True
            n = 0
            if must_copy:
                self.r_a = n
            self.inst_ptr = 0
            while self.inst_ptr < len(self.program):
                opcode = self.program[self.inst_ptr]
                operand = self.program[self.inst_ptr + 1]
                ret = self.inst_set[opcode](operand)
                if ret != 1:
                    self.inst_ptr += 2
                if must_copy and (self.stdout != self.program[:len(self.stdout)]):
                    n += 1
                    copy_success = False
                    break
            if must_copy and not copy_success:
                continue
            break
    
    def _dv(self, op: int) -> int:
        op = self.get_combo_op(op)
        return self.r_a // (2 ** op)
    
    def adv(self, op: int):
        self.r_a = self._dv(op)

    def bdv(self, op: int):
        self.r_b = self._dv(op)

    def cdv(self, op: int):
        self.r_c = self._dv(op)
    
    def bxl(self, op: int):
        self.r_b = self.r_b ^ op

    def bst(self, op: int):
        op = self.get_combo_op(op)
        self.r_b = op % 8

    def jnz(self, op: int) -> int | None:
        if self.r_a == 0:
            return
        else:
            self.inst_ptr = op
            return 1

    def bxc(self, _):
        self.r_b = self.r_b ^ self.r_c

    def out(self, op: int):
        op = self.get_combo_op(op)
        self.stdout.append(op % 8)

In [ ]:
# puzzle_part = int(sys.argv[1])
puzzle_part = 1
sample = readutf8('sample.txt')
puzzle = readutf8('input.txt')

In [ ]:
cpu = ElfComputer.from_str(puzzle)

In [ ]:
cpu.run_program(must_copy=True if puzzle_part == 2 else False)

In [ ]:
','.join(map(str, cpu.stdout))